对抗验证

In [ ]:
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import lightgbm as lgb


def get_adv_feats(df_train, df_test, feats):
    df_train['adv'] = 1
    df_test['adv'] = 0
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    params = {
        'learning_rate': 0.1,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'seed': 2222,
        'n_jobs': 4,
        'verbose': -1,
    }

    fold_num = 5
    seeds = [2023]
    new_feats = []

    for f in feats:
        oof = np.zeros(len(df))
        for seed in seeds:
            kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
            for fold, (train_index, val_index) in enumerate(kf.split(df[[f]], df['adv'])):
                train_set = lgb.Dataset(df.loc[train_index, [f]], df.loc[train_index, ['adv']])
                val_set = lgb.Dataset(df.loc[val_index, [f]], df.loc[val_index, ['adv']])
                model = lgb.train(params, train_set, valid_sets=val_set, num_boost_round=10000,
                                  callbacks=[lgb.early_stopping(100), lgb.log_evaluation(-1)])
                oof[val_index] += model.predict(df.loc[val_index, [f]]) / len(seeds)
                score = auc(df.loc[val_index, 'adv'], oof[val_index])
                if score > 0.85:
                    print('--------------------------------------', f, score)
                else:
                    new_feats.append(f)
                break
    return new_feats